In [ ]:
from flask import Flask, request, render_template_string
import threading
import mysql.connector

app = Flask(__name__)

html_template = '''
<!doctype html>
<html lang="en">
<head>
  <title>ورودی کاربر</title>
</head>
<body>
  <h2>مقادیر را وارد کنید:</h2>
  <form method="POST">
    h: <input type="number" name="h"><br>
    t: <input type="number" name="t"><br>
    <input type="submit" value="ارسال">
  </form>

  {% if result is not none %}
    <h3>نتیجه: {{ result }}</h3>
  {% endif %}
  
  {% if error %}
    <h3 style="color:red;">خطا: {{ error }}</h3>
  {% endif %}
</body>
</html>
'''

def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="developer",
        password="",  # رمزتو اینجا بزن
        database="love"
    )

@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    error = None
    if request.method == 'POST':
        h = request.form.get('h', '0')
        t = request.form.get('t', '0')
        try:
            h = int(h)
            t = int(t)
            conn = get_db_connection()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO ali (h, t) VALUES (%s, %s)", (h, t))
            conn.commit()

            total = h + t

            # فقط برای سادگی، بروزرسانی روی آخرین ردیف
            cursor.execute("SELECT LAST_INSERT_ID()")
            last_id = cursor.fetchone()[0]
            cursor.execute("UPDATE ali SET result = %s WHERE id = %s", (total, last_id))
            conn.commit()

            cursor.close()
            conn.close()
            result = total
        except Exception as e:
            error = str(e)
            print("Error:", error)

    return render_template_string(html_template, result=result, error=error)

def run_app():
    app.run(debug=False, use_reloader=False)

import threading
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/May/2025 23:23:03] "GET / HTTP/1.1" 200 -
[2025-05-20 23:23:06,551] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\Users\abolfazl\anaconda3\Lib\site-packages\mysql\connector\connection_cext.py", line 772, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Unknown column 'result' in 'field list'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\abolfazl\anaconda3\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abolfazl\anaconda3\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abolfazl\anaconda3\Lib\site-packages\flask\app.py", line 880, in full_dispatch_reques